Big thanks to Jiwei Liu for Augment insight!
https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment


In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np

from datetime import datetime
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

In [2]:
train=pd.read_csv("train.csv").drop("ID_code",axis=1)
test=pd.read_csv("test.csv").drop("ID_code",axis=1)

In [3]:
train['target'].value_counts()

0    179902
1     20098
Name: target, dtype: int64

In [7]:
## Inspiration from
#https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment
def augment(train,num_n=1,num_p=2):
    newtrain=[train]
    
    n=train[train.target==0]
    for i in range(num_n):
        newtrain.append(n.apply(lambda x:x.values.take(np.random.permutation(len(n)))))
    
    for i in range(num_p):
        p=train[train.target>0]
        newtrain.append(p.apply(lambda x:x.values.take(np.random.permutation(len(p)))))
    return pd.concat(newtrain)
#df=oversample(train,2,1)

In [8]:
param = {
   "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    "num_leaves" : 13,
    "learning_rate" : 0.01,
    "bagging_freq": 5,
    "bagging_fraction" : 0.4,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    "bagging_seed" : 10,
    "verbosity" : 1,
}

In [9]:
result=np.zeros(test.shape[0])

rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5,random_state=10)
for counter,(train_index, valid_index) in enumerate(rskf.split(train, train.target),1):
    print (counter)
    
    #Train data
    t=train.iloc[train_index]
    t=augment(t)
    trn_data = lgb.Dataset(t.drop("target",axis=1), label=t.target)
    
    #Validation data
    v=train.iloc[valid_index]
    val_data = lgb.Dataset(v.drop("target",axis=1), label=v.target)
    
    #Training
    model = lgb.train(param, trn_data, 1000000, valid_sets = [val_data], verbose_eval=500, early_stopping_rounds = 3000)
    result +=model.predict(test)


1
Training until validation scores don't improve for 3000 rounds.
[500]	valid_0's auc: 0.871496
[1000]	valid_0's auc: 0.879061
[1500]	valid_0's auc: 0.883554
[2000]	valid_0's auc: 0.887281
[2500]	valid_0's auc: 0.889676
[3000]	valid_0's auc: 0.89222
[3500]	valid_0's auc: 0.893727
[4000]	valid_0's auc: 0.895135
[4500]	valid_0's auc: 0.896111
[5000]	valid_0's auc: 0.896948
[5500]	valid_0's auc: 0.897571
[6000]	valid_0's auc: 0.898212
[6500]	valid_0's auc: 0.898675
[7000]	valid_0's auc: 0.898982
[7500]	valid_0's auc: 0.899143
[8000]	valid_0's auc: 0.899252
[8500]	valid_0's auc: 0.899385
[9000]	valid_0's auc: 0.899483
[9500]	valid_0's auc: 0.899487
[10000]	valid_0's auc: 0.899551
[10500]	valid_0's auc: 0.899613
[11000]	valid_0's auc: 0.899679
[11500]	valid_0's auc: 0.899682
[12000]	valid_0's auc: 0.899683
[12500]	valid_0's auc: 0.8997
[13000]	valid_0's auc: 0.899675
[13500]	valid_0's auc: 0.899647
[14000]	valid_0's auc: 0.899588
[14500]	valid_0's auc: 0.89956
[15000]	valid_0's auc: 0.89947

[4000]	valid_0's auc: 0.899659
[4500]	valid_0's auc: 0.900609
[5000]	valid_0's auc: 0.901434
[5500]	valid_0's auc: 0.901987
[6000]	valid_0's auc: 0.902456
[6500]	valid_0's auc: 0.902774
[7000]	valid_0's auc: 0.903068
[7500]	valid_0's auc: 0.903347
[8000]	valid_0's auc: 0.903519
[8500]	valid_0's auc: 0.903641
[9000]	valid_0's auc: 0.90375
[9500]	valid_0's auc: 0.903834
[10000]	valid_0's auc: 0.903828
[10500]	valid_0's auc: 0.903781
[11000]	valid_0's auc: 0.903758
[11500]	valid_0's auc: 0.903806
[12000]	valid_0's auc: 0.903785
[12500]	valid_0's auc: 0.903776
Early stopping, best iteration is:
[9812]	valid_0's auc: 0.903848
10
Training until validation scores don't improve for 3000 rounds.
[500]	valid_0's auc: 0.868868
[1000]	valid_0's auc: 0.87687
[1500]	valid_0's auc: 0.881112
[2000]	valid_0's auc: 0.884844
[2500]	valid_0's auc: 0.887389
[3000]	valid_0's auc: 0.88955
[3500]	valid_0's auc: 0.891433
[4000]	valid_0's auc: 0.89281
[4500]	valid_0's auc: 0.893873
[5000]	valid_0's auc: 0.89481

[10000]	valid_0's auc: 0.901368
[10500]	valid_0's auc: 0.90142
[11000]	valid_0's auc: 0.901453
[11500]	valid_0's auc: 0.901451
[12000]	valid_0's auc: 0.901474
[12500]	valid_0's auc: 0.901486
[13000]	valid_0's auc: 0.901551
[13500]	valid_0's auc: 0.901578
[14000]	valid_0's auc: 0.901526
[14500]	valid_0's auc: 0.901464
[15000]	valid_0's auc: 0.901404
[15500]	valid_0's auc: 0.901413
[16000]	valid_0's auc: 0.901389
[16500]	valid_0's auc: 0.901299
Early stopping, best iteration is:
[13712]	valid_0's auc: 0.901584
18
Training until validation scores don't improve for 3000 rounds.
[500]	valid_0's auc: 0.871747
[1000]	valid_0's auc: 0.879574
[1500]	valid_0's auc: 0.884144
[2000]	valid_0's auc: 0.887729
[2500]	valid_0's auc: 0.890348
[3000]	valid_0's auc: 0.892472
[3500]	valid_0's auc: 0.894262
[4000]	valid_0's auc: 0.895626
[4500]	valid_0's auc: 0.896728
[5000]	valid_0's auc: 0.897506
[5500]	valid_0's auc: 0.898083
[6000]	valid_0's auc: 0.898539
[6500]	valid_0's auc: 0.898861
[7000]	valid_0's 

[12000]	valid_0's auc: 0.900132
[12500]	valid_0's auc: 0.900127
[13000]	valid_0's auc: 0.900099
[13500]	valid_0's auc: 0.900069
[14000]	valid_0's auc: 0.900014
[14500]	valid_0's auc: 0.899987
[15000]	valid_0's auc: 0.899957
Early stopping, best iteration is:
[12070]	valid_0's auc: 0.900143


In [11]:
submission = pd.read_csv('sample_submission.csv')
submission['target'] = result/counter
#filename="{:%Y-%m-%d_%H_%M}_sub.csv".format(datetime.now())
submission.to_csv('submission_with_augm.csv', index=False)